<center><p float="center">
  <img src="https://upload.wikimedia.org/wikipedia/commons/e/e9/4_RGB_McCombs_School_Brand_Branded.png" width="300" height="100"/>
  <img src="https://mma.prnewswire.com/media/1458111/Great_Learning_Logo.jpg?p=facebook" width="200" height="100"/>
</p></center>

<center><font size=10>Generative AI for Business Applications</center></font>
<center><font size=6>Fine-Tunning LLMs - Week 1</center></font>

<center><p float="center">
  <img src="https://images.pexels.com/photos/5699431/pexels-photo-5699431.jpeg" width=720></a>
<center><font size=6>Fine-Tuned AI for Summarizing Medical Conversations</center></font>

# **Problem Statement**

## **Business Context**

In today’s fast-paced healthcare environment, doctors often struggle with the time-consuming task of documenting patient consultations. These conversations are usually long, unstructured, and difficult to review later, which makes writing and editing notes a heavy burden. The inefficiencies carry over into follow-ups, as important details may be buried within lengthy transcripts. To address this challenge, a healthcare technology team is developing an LLM-powered assistant designed to streamline clinical documentation. This AI assistant will automatically generate structured summaries, highlight key patient details, and organize consultation notes for easy review. By applying this solution to real-world consultation data, the team aims to demonstrate how advanced language models can reduce administrative workload, minimize errors, and ultimately improve the quality of patient care.

##  **Objective**

The goal is to develop an AI-powered system that demonstrates how Natural Language Processing (NLP) can support doctors by automatically generating clear and structured consultation notes from patient interactions.

Specifically, the system aims to:

* Capture patient concerns, including symptoms, medical history, and lifestyle factors.
* Summarize the doctor’s findings and recommendations, such as diagnoses, prescriptions, tests, and follow-up actions.
* Ensure that all notes maintain a consistent, professional clinical tone.
* Improve accuracy and reliability by fine-tuning on domain-specific medical dialogues.

This case study focuses on building a prototype that transforms unstructured doctor-patient conversations into concise, structured clinical summary. By doing so, it reduces the time spent on manual documentation, minimizes the risk of missed information, and enhances the overall quality of patient care.

Through successful implementation, the organization seeks to reduce administrative workload, support more efficient follow-ups, and ultimately improve healthcare outcomes.


## **Data Description**

The dataset is divided into three files: training, validation, and test. It consists of two primary columns:

1. **Conversation** - Contains the raw transcripts of doctor-patient dialogues, which are often long and unstructured.

2. **Summary** - Provides the corresponding concise and structured clinical summary, making it suitable for training supervised summarization models.


# **Installing and Importing Necessary Libraries**

In [ ]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.32.post2 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf huggingface_hub hf_transfer
!pip install transformers==4.51.3
!pip install --no-deps unsloth
!pip install -q datasets evaluate bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.8/184.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 104.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.2
    Uninstalling transformers-4.55.2:
      Successfully uninstalled transformers-4.55.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.8.8 requires msgspec, which is not installed.
unsloth-zoo 2025.8.8 requires tyro, which is not installed.
unsloth-zoo 2025.8.8 requires datasets<4.0.0,>=3.4.1, but you have datasets 4.0.0 which is incompatible.
     ━━━━━━━━━━━━

**Note**:
- After running the above cell, kindly restart the runtime (for Google Colab) or notebook kernel (for Jupyter Notebook), and run all cells sequentially from the next cell.
- On executing the above line of code, you might see a warning regarding package dependencies. This error message can be ignored as the above code ensures that all necessary libraries and their dependencies are maintained to successfully execute the code in ***this notebook***.

In [ ]:
from unsloth import FastLanguageModel
import torch
import evaluate
from tqdm import tqdm
import pandas as pd
from datasets import Dataset

from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


# **1. Evaluation of LLM before FineTuning**

Before investing time and resources in fine-tuning, it is important to measure **how well the base model performs "as is."**
This gives us a baseline score, which we will later compare against fine-tuned performance to demonstrate improvement.

### **Loading the Testing Data**


In [ ]:
testing_data = pd.read_csv("/content/finetuning_medical_testing.csv")           # Load medical test dataset containing dialogues and gold summaries

test_dialogues = testing_data['conversation'].tolist()                          # dialogues (inputs)
test_summaries = testing_data['summary'].tolist()                               # human-written summaries (ground truth)

### **Loading the Mistral Model**

We use the **Mistral-7B Instruct model** (7 billion parameter model optimized for instructions).
It is loaded with **4-bit quantization** - meaning it runs efficiently on limited GPU memory without losing much accuracy.


In [ ]:
# Load the instruction-tuned Mistral 7B model with 4-bit quantization
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/mistral-7b-instruct-v0.2-bnb-4bit",                     # model name
    max_seq_length=4096,                                                        # maximum sequence length
    dtype=None,                                                                 # auto-select data type
    load_in_4bit=True                                                           # load in 4-bit for memory efficiency
)

==((====))==  Unsloth 2025.8.9: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
FastLanguageModel.for_inference(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): Mis

### **Generate Summaries**


We now ask the model to summarize dialogues **without any fine-tuning**.
A custom prompt template is used to ensure the model follows instructions consistently.

In [ ]:
alpaca_prompt_template = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Write a concise summary of the following dialogue.

### Input:
{}

### Response:
{}
"""

We will create a function to manage the entire prediction process, which includes the following steps:
1. Structure the Prompt
2. Tokenize the Prompt
3. Generate the Output from Tokens
4. Decode the Generated Output

This function allows us to avoid code repetition and streamline the process. Whenever we need to test a model, we can simply call this function instead of rewriting these steps each time.


In [ ]:
def generate_summaries(dialogues, model, tokenizer, prompt_template, max_new_tokens=100):
    summaries = []

    for dialogue in dialogues:
        # create a prompt for the dialogue
        prompt = prompt_template.format(dialogue," ")

        # tokenize the prompt
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        # generate the output
        outputs = model.generate(**inputs,
                                 max_new_tokens=max_new_tokens,
                                 pad_token_id=tokenizer.eos_token_id
                                 )

        # decode: skip input tokens, keep only generated part
        input_len = inputs["input_ids"].shape[-1]
        summary = tokenizer.decode(outputs[0][input_len:],
                                   skip_special_tokens=True,
                                   cleanup_tokenization_spaces=True
                                   )

        summaries.append(summary.strip())

    return summaries

In [ ]:
# Calling the function to generate summaries
predicted_summaries = generate_summaries(dialogues=test_dialogues,
                                         model=model,
                                         tokenizer=tokenizer,
                                         prompt_template=alpaca_prompt_template
                                         )

### **Evaluate Using BERTScore**


**BERTScore** is an evaluation metric for text generation tasks such as summarization, translation, and captioning. Unlike traditional metrics like ROUGE or BLEU that rely on exact word overlaps, BERTScore uses embeddings from a pre-trained BERT model to measure **semantic similarity** between the generated text (predictions) and the human-written text (references). This makes it more robust in capturing meaning, even when different words are used.

* **Precision** - Measures how much of the content in the generated text is actually relevant to the reference. A high precision means the model is not adding irrelevant or “extra” information.

* **Recall** - Measures how much of the important content from the reference is captured by the generated text. A high recall means the model covers most of the key points, even if it includes some extra details.

* **F1 Score** - Combines both precision and recall into a balanced score. It shows how well the generated text both *covers the important content* and *stays relevant*. This is usually reported as the main metric for BERTScore.

In short, BERTScore helps evaluate not just word matching but whether the **meaning** of the generated text aligns with the reference.


In [ ]:
bert_scorer = evaluate.load("bertscore")                                        # Load BERTScore evaluation metric

score = bert_scorer.compute(                                                    # Compute BERTScore between generated summaries and gold references
    predictions=predicted_summaries,                                            # model-generated summaries
    references=test_summaries,                                                  # ground-truth summaries
    lang='en',                                                                  # language
    rescale_with_baseline=True                                                  # normalize scores for fair comparison
)


baseline_score = sum(score['f1']) / len(score['f1'])                            # Compute the average F1 score across test set
print(baseline_score)                                                           # Print the average F1 score

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0.37641953192651273


**The BERT Score of Mistral LLM is 0.37**


# **2. FineTuning an LLM**

Now that we have measured the **baseline performance** of the Mistral-7B model, the next step is to **fine-tune the model on our medical dataset.**

Fine-tuning helps the model specialize in summarizing medical dialogues, improving relevance and accuracy beyond its general-purpose training.

### **Load and Prepare Training Data**

We begin with the **training dataset**, which contains doctor-patient conversations paired with gold-standard summaries.

We then convert the dataset into a Hugging Face `Dataset` format for efficient handling.

In [ ]:
# Load training dataset
training = pd.read_csv("/content/finetuning_medical_training.csv")
training_dict = training.to_dict(orient='list')

# Create a dataset from the dictionary
training_dataset = Dataset.from_dict(training_dict)

In [ ]:
# Instruction-tuned prompt template for summarization
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

To make training effective, we format each example into this structure and append an EOS token to clearly signal the end of the response.

In [ ]:
EOS_TOKEN = tokenizer.eos_token

In [ ]:
# Format each training example into the Alpaca prompt structure
def prompt_formatter(example, prompt_template):
    instruction='Write a concise summary of the following dialogue.'
    dialogue=example["conversation"]
    summary=example["summary"]

    formatted_prompt = prompt_template.format(instruction, dialogue, summary) + EOS_TOKEN

    return {'text': formatted_prompt}

In [ ]:
# Apply formatting to training dataset
formatted_training_dataset = training_dataset.map(
    prompt_formatter,
    fn_kwargs={'prompt_template': alpaca_prompt}
)

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

### **Load and Prepare Validation Data**

We also prepare a **validation dataset** (unseen during training).
This allows us to track the model's progress and prevent overfitting.

In [ ]:
# Load validation dataset
validation=pd.read_csv("/content/finetuning_medical_validation.csv")
validation_dict =validation.to_dict(orient='list')

# Create a dataset from the dictionary
validation_dataset = Dataset.from_dict(validation_dict)

In [ ]:
# Apply formatting to training dataset
formatted_validation_dataset = validation_dataset.map(
    prompt_formatter,
    fn_kwargs={'prompt_template': alpaca_prompt}
)

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

### **Apply Parameter-Efficient Fine-Tuning (LoRA)**

We now patch in the adapter modules to the base model using the `get_peft_model` method.

In [ ]:
# Apply LoRA fine-tuning (PEFT) to the base model
model = FastLanguageModel.get_peft_model(
    model,                                                                      # base model to fine-tune
    r=16,                                                                       # rank of LoRA matrices
    lora_alpha=16,                                                              # scaling factor for LoRA updates
    lora_dropout=0,                                                             # dropout (0 means no dropout)
    bias="none",                                                                # don't fine-tune bias terms
    target_modules=[                                                            # layers where LoRA is applied
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    use_gradient_checkpointing=True,                                            # saves memory by recomputing activations
    random_state=42,                                                            # reproducibility
    loftq_config=None                                                           # not using LoFTQ quantization here
)

Unsloth 2025.8.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


First, we load the model using `FastLanguageModel`. After initializing the base model, we patch the adapter models using `get_peft_model`. This allows us to enhance the model by integrating LoRA (Low-Rank Adaptation) weights, which are now displayed as part of the model's configuration. This setup facilitates improved customization and adaptability of the model for specialized tasks.

In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): l

### **Configure Training**

We now define training arguments for **supervised fine-tuning (SFT)** using Hugging Face's `SFTTrainer`.
The setup ensures training runs efficiently on Colab GPUs.

In [ ]:
# Supervised Fine-Tuning (SFT) trainer setup
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_training_dataset,
    eval_dataset = formatted_validation_dataset,
    dataset_text_field = "text",                                                # column containing the text
    max_seq_length = 2048,                                                      # max sequence length
    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer),                # handles padding, batching
    dataset_num_proc = 2,                                                       # parallel dataset processing
    packing = False,                                                            # set True to pack short sequences (faster)

    # Training hyperparameters
    args = TrainingArguments(
        per_device_train_batch_size = 2,                                        # batch size per GPU
        gradient_accumulation_steps = 4,                                        # simulate larger batch via accumulation
        warmup_steps = 5,                                                       # warmup before LR schedule
        max_steps = 60,                                                         # limit training steps (overrides epochs)
        learning_rate = 2e-4,                                                   # base learning rate
        fp16 = not is_bfloat16_supported(),                                     # use fp16 if bf16 not available
        bf16 = is_bfloat16_supported(),                                         # use bf16 if supported (better precision)
        logging_steps = 1,                                                      # log every step
        optim = "adamw_8bit",                                                   # memory-efficient optimizer
        weight_decay = 0.01,                                                    # weight decay for regularization
        lr_scheduler_type = "linear",                                           # linear learning rate decay
        seed = 3407,                                                            # reproducibility
        output_dir = "outputs",                                                 # save model outputs
        report_to = "none"                                                      # disable logging to external services
    ),
)


Unsloth: Tokenizing ["text"]:   0%|          | 0/60 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"]:   0%|          | 0/20 [00:00<?, ? examples/s]

In [ ]:
# Begin fine-tuning
training_history = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 60 | Num Epochs = 9 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 7,283,675,136 (0.58% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.374300
2,2.308200
3,1.830800
4,1.561200
5,1.188600
6,0.876100
7,0.757700
8,0.652000
9,0.513700
10,0.547000


### **Save Fine-Tuned Model**


After training, we save the **LoRA fine-tuned model** for future inference.

In [ ]:
# Save locally
lora_model_name = "finetuned_mistral_llm"
model.save_pretrained(lora_model_name)                                          # Save LoRA-finetuned model

# Verify saved files
!ls -lh {lora_model_name}                                                       # List files inside the saved model folder

total 161M
-rw-r--r-- 1 root root  950 Aug 26 08:45 adapter_config.json
-rw-r--r-- 1 root root 161M Aug 26 08:45 adapter_model.safetensors
-rw-r--r-- 1 root root 5.2K Aug 26 08:45 README.md


In [ ]:
# # Comment out this cell if you want to save the model to Drive

# from google.colab import drive
# drive.mount('/content/drive')

# drive_model_path = "/content/drive/MyDrive/finetuned_mistral_llm"

# !cp -r {lora_model_name} {drive_model_path}

# **3. Evaluation of LLM after FineTuning**

Once the model has been fine-tuned, the next step is to **measure its effectiveness**. In business terms, this is like checking whether the training investment has paid off and whether the system can generate outputs that align with expectations.

### **Loading the Finetuned Mistral Model**

In [ ]:
finetuned_model, finetuned_tokenizer = FastLanguageModel.from_pretrained(
    model_name=lora_model_name,                                                 # Replace the model name with "drive_model_path" if you are loading the model from Drive
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True
)

==((====))==  Unsloth 2025.8.9: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


> **Note** <br> It is not strictly necessary to save and reload the model if you are evaluating it in the same Colab session - the model is still in memory. **However, as a best practice,** we save the fine-tuned model and then reload it for evaluation.

### **Generate Summaries**

In [ ]:
# Calling the function to generate summaries
predicted_summaries = generate_summaries(dialogues=test_dialogues,
                                         model=finetuned_model,
                                         tokenizer=finetuned_tokenizer,
                                         prompt_template=alpaca_prompt_template
                                         )

### **Evaluate Using BERTScore**

In [ ]:
score = bert_scorer.compute(
    predictions=predicted_summaries,
    references=test_summaries,
    lang='en',
    rescale_with_baseline=True
)
finetune_score = sum(score['f1']) / len(score['f1'])
print(finetune_score)

0.6874006405472756


**The BERT Score of Finetuned Mistral LLM is 0.67**

- This suggests that the model is producing reasonably good summaries that align well with human expectations.

- While not perfect, it demonstrates that fine-tuning has meaningfully improved the model's ability to perform the summarization task.

# **Conclusion**


* The fine-tuned model generates concise and clinically accurate summaries, enhancing the quality of documentation and supporting improved patient care.

* Summaries now align closely with clinical communication needs, allowing medical practitioners quick access to relevant and structured information.

* This approach establishes a foundation for developing intelligent healthcare support systems that are context-aware, scalable, and tailored to the needs of medical professionals.

* The system's effectiveness is contingent on the availability of high-quality task-relevant consultation data and may require further refinement for diverse healthcare contexts.

* While the model produces clinically relevant summaries, continuous evaluation and possible adjustments are necessary to maintain alignment with evolving healthcare standards and practices.

<font size = 6 color="#4682B4"><b> Power Ahead </font>
___